## Problem Set 2: Job Search

A Python solution to the finite life Job Search problem.

We start with some imports

In [151]:
import numpy as np
import quantecon as qe

from numba import njit, prange
import matplotlib.pyplot as plt

To keep the code simple, some variables will be global.

In [152]:
T = 65
w_vals = (7, 10, 11)
p = (0.25, 0.5, 0.25)
p_cdf = np.cumsum(p)

Here's a little function to draw from $p$

In [153]:
@njit
def draw_from_offer_distribution():
    return qe.random.draw(p_cdf)


The next function computes values and policies.

The possible wage values are $w_0, w_1, w_2$

The value functions $v_t(w)$ are returned as a matrix $V$ of the form

$$
V = 
\begin{pmatrix}
    v_0(w_0) & v_0(w_1) & v_0(w_2) \\
    v_1(w_0) & v_1(w_1) & v_1(w_2) \\
     & \vdots & \\
    v_t(w_0) & v_t(w_1) & v_t(w_2) \\
     & \vdots & 
\end{pmatrix}
$$

The policy functions, which give the optimal choice at each state and time, have the interpretation

$$ \sigma_t(w) =
\begin{cases}
    1 & \text{if accept $w$ at time $t$} \\
    0 & \text{otherwise}
\end{cases}
$$

The policy functions are returned as a matrix $\sigma$ of the form

$$
\sigma = 
\begin{pmatrix}
    \sigma_0(w_0) & \sigma_0(w_1) & \sigma_0(w_2) \\
    \sigma_1(w_0) & \sigma_1(w_1) & \sigma_1(w_2) \\
     & \vdots & \\
    \sigma_t(w_0) & \sigma_t(w_1) & \sigma_t(w_2) \\
     & \vdots & 
\end{pmatrix}
$$

In [154]:
@njit
def compute_values_and_policies(β=0.96, c=8):

    n = len(w_vals) 
    V = np.empty((T, n))
    σ = np.zeros((T, n), dtype=np.int64)

    t = T-1
    for i, w in enumerate(w_vals):
        V[t, i] = max(c, w) 
        σ[t, i] = (w > c)

    while t > 0:
        for i, w in enumerate(w_vals):
            
            accept_val = w * (1 - β**(T-t+1)) / (1 - β)
            EV = 0.0
            for j in range(n):
                EV += V[t, j] * p[j]
            continue_val = c + β * EV
            
            if accept_val > continue_val:
                σ[t-1, i] = 1
                V[t-1, i] = accept_val
            else:
                V[t-1, i] = continue_val
                
        t -= 1

    return V, σ

Here's jitted code that simulates the life of one agent, recording the wage that they accept at and their age when they accept.

In [166]:
@njit
def sim_life(σ):
    for t in range(T):
        i = draw_from_offer_distribution()
        if σ[t, i] == 1:
            accepted_wage = w_vals[i]
            start_work_age = t
            return accepted_wage, start_work_age+1
        
    start_work_age = T
    accepted_wage = 0
    return accepted_wage, start_work_age

Here's a function that simulates the lives of many agents and takes an average over their statistics.

Parallelization is not implemented because the individual calculations are so short.

In [167]:
@njit
def sim_stats(σ, m=1_000_000):
    wage_mean = 0
    age_mean = 0
    for i in range(m):
        accepted_wage, start_work_age = sim_life(σ)
        age_mean += start_work_age
        wage_mean += accepted_wage
    return wage_mean / m, age_mean / m
        

Now let's actually compute the optimal policies and simulate some agents to calculate statistics.

In [178]:

V, σ = compute_values_and_policies()

In [179]:
%%time

wage_mean, age_mean = sim_stats(σ)

CPU times: user 153 ms, sys: 0 ns, total: 153 ms
Wall time: 151 ms


In [180]:
wage_mean

11.0

In [181]:
age_mean

3.999721

Total time to simulate for 1,000,000 agents is a few hundred ms on my machine.

What happens when we change $c$?

In [182]:

V, σ = compute_values_and_policies(c=4.5)

In [183]:
%%time

wage_mean, age_mean = sim_stats(σ)

CPU times: user 103 ms, sys: 0 ns, total: 103 ms
Wall time: 101 ms


In [184]:
wage_mean

10.71835

In [185]:
age_mean

2.876221